In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pytest
import unittest
from pandas.testing import assert_frame_equal

from mcda.mcda_run import main
from mcda.utils import *
from mcda.configuration.config import Config
from mcda.utility_functions.normalization import Normalization

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## minmax

In [2]:
input_matrix = read_matrix("tests/resources/input_matrix_without_uncert.csv")

In [3]:
#input_matrix['ind1'] = [3,3,3,3]

In [4]:
input_matrix

,alternatives,ind1,ind2,ind3,ind4,ind5,ind6
0,alt1,-15.2,8.2,0.04,0.02,24.5,6.20
1,alt2,-12.4,8.7,0.05,0.02,24.5,4.80
2,alt3,1.6,2.0,0.11,0.14,14.0,0.60
3,alt4,-39.7,14.0,0.01,0.02,26.5,4.41


In [5]:
imd = input_matrix.drop(input_matrix.columns[0],axis=1)

In [6]:
imd

,ind1,ind2,ind3,ind4,ind5,ind6
0,-15.2,8.2,0.04,0.02,24.5,6.20
1,-12.4,8.7,0.05,0.02,24.5,4.80
2,1.6,2.0,0.11,0.14,14.0,0.60
3,-39.7,14.0,0.01,0.02,26.5,4.41


#### identify positive and negative polarities

In [7]:
polarities = ("-","-","+","+","+","+")

In [8]:
indeces_plus = [i for i, e in enumerate(polarities) if e == "+"]
indeces_minus = [i for i, e in enumerate(polarities) if e == "-"]

In [9]:
print(type(indeces_plus))
print(indeces_minus)

<class 'list'>
[0, 1]


In [10]:
# from input matrix select columns that go into minmax and the ones that go into reversed minmax
plus_pol = imd.iloc[:, indeces_plus]
minus_pol = imd.iloc[:, indeces_minus]

In [11]:
plus_pol

,ind3,ind4,ind5,ind6
0,0.04,0.02,24.5,6.20
1,0.05,0.02,24.5,4.80
2,0.11,0.14,14.0,0.60
3,0.01,0.02,26.5,4.41


In [12]:
minus_pol

,ind1,ind2
0,-15.2,8.2
1,-12.4,8.7
2,1.6,2.0
3,-39.7,14.0


In [13]:
x_plus = plus_pol.values # returns a numpy array
x_minus = minus_pol.values
x_minus

array([[-15.2,   8.2],
       [-12.4,   8.7],
       [  1.6,   2. ],
       [-39.7,  14. ]])

In [14]:
x_minus_test = minus_pol.to_numpy()
x_minus_test

array([[-15.2,   8.2],
       [-12.4,   8.7],
       [  1.6,   2. ],
       [-39.7,  14. ]])

In [15]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0.1, 1), copy=False)
x_scaled = min_max_scaler.fit_transform(x_plus)

In [16]:
x_scaled

array([[0.37      , 0.1       , 0.856     , 1.        ],
       [0.46      , 0.1       , 0.856     , 0.775     ],
       [1.        , 1.        , 0.1       , 0.1       ],
       [0.1       , 0.1       , 1.        , 0.71232143]])

In [17]:
indicators_scaled_minmax_plus = pd.DataFrame(x_scaled)

In [18]:
indicators_scaled_minmax_plus

,0,1,2,3
0,0.37,0.1,0.856,1.000000
1,0.46,0.1,0.856,0.775000
2,1.00,1.0,0.100,0.100000
3,0.10,0.1,1.000,0.712321


In [19]:
def reversed_minmax_scaler(data):
    """
    Rescales the given data to the range [0, 1] in a reversed scale
    where the smallest the value the better, using MinMaxScaler.

    Args:
    - data: A list or numpy array of numeric data to be scaled.

    Returns:
    - scaled_data: The scaled data as a numpy array.
    """
    import numpy as np
    
    # Convert data to numpy array
    data = np.array(data)
    print(data)
    print('-----')
    
    # Calculate the maximum and minimum values
    max_val = np.max(data)
    max_val_test = np.max(data, axis=0)
    
    print(max_val)
    print('-----')
    print(max_val_test)
    print('-----')
    
    min_val = np.min(data)
    min_val_test = np.min(data, axis=0)
    
    # Rescale the data to the range [0, 1]
    scaled_data = (data - min_val) / (max_val - min_val)
    scaled_data_test = (data - min_val_test) / (max_val_test - min_val_test)*(1-0.1) + 0.1
    
    print(scaled_data)
    print('-----')
    print(scaled_data_test)
    
    return scaled_data_test


In [20]:
indicators_scaled_minmax_minus = reversed_minmax_scaler(x_minus)

[[-15.2   8.2]
 [-12.4   8.7]
 [  1.6   2. ]
 [-39.7  14. ]]
-----
14.0
-----
[ 1.6 14. ]
-----
[[0.45623836 0.89199255]
 [0.50837989 0.90130354]
 [0.76908752 0.77653631]
 [0.         1.        ]]
-----
[[0.63389831 0.565     ]
 [0.69491525 0.6025    ]
 [1.         0.1       ]
 [0.1        1.        ]]


In [21]:
indicators_scaled_minmax_minus = pd.DataFrame(indicators_scaled_minmax_minus)

In [22]:
indicators_scaled_minmax_minus

,0,1
0,0.633898,0.5650
1,0.694915,0.6025
2,1.000000,0.1000
3,0.100000,1.0000


In [23]:
indicators_scaled_minmax = pd.DataFrame(index=range(4),columns=range(6))
indicators_scaled_minmax.shape

(4, 6)

In [24]:
indicators_scaled_minmax_plus.iloc[:, 1]

0    0.1
1    0.1
2    1.0
3    0.1
Name: 1, dtype: float64

In [25]:
# merge back the plus and minus together
indicators_scaled_minmax = pd.DataFrame(columns=range(6))
for i,index_p in enumerate(indeces_plus): indicators_scaled_minmax.iloc[:, index_p] = indicators_scaled_minmax_plus.iloc[:, i]
for j, index_n in enumerate(indeces_minus): indicators_scaled_minmax.iloc[:, index_n]=indicators_scaled_minmax_minus.loc[:,j]

/var/folders/mt/lx9bxf895gq_bfbn6l6_m3kr0000gn/T/ipykernel_59657/3699407345.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  for i,index_p in enumerate(indeces_plus): indicators_scaled_minmax.iloc[:, index_p] = indicators_scaled_minmax_plus.iloc[:, i]
/var/folders/mt/lx9bxf895gq_bfbn6l6_m3kr0000gn/T/ipykernel_59657/3699407345.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  for j, index_n in enumerate(indeces_minus): indicators_scaled_minmax.iloc[:, index_n]=indicators_scaled_minmax_minus.loc[:,j]


In [26]:
indicators_scaled_minmax

,0,1,2,3,4,5
0,0.633898,0.5650,0.37,0.1,0.856,1.000000
1,0.694915,0.6025,0.46,0.1,0.856,0.775000
2,1.000000,0.1000,1.00,1.0,0.100,0.100000
3,0.100000,1.0000,0.10,0.1,1.000,0.712321


In [29]:
# test class
norm = Normalization(imd)
test_res_01 = norm.minmax(polarities, feature_range=(0.1, 1))
test_res_01

,0,1,2,3,4,5
0,0.633898,0.5650,0.37,0.1,0.856,1.000000
1,0.694915,0.6025,0.46,0.1,0.856,0.775000
2,1.000000,0.1000,1.00,1.0,0.100,0.100000
3,0.100000,1.0000,0.10,0.1,1.000,0.712321


In [28]:
# if you save the file it might compromise the unit tests!
#indicators_scaled_minmax.to_csv('tests/resources/normalization/res_minmax_pol_no0.csv',sep=';',decimal='.') 

## target

In [ ]:
imd

In [ ]:
imd['ind1'] = [-1,-3,-3,0]

In [ ]:
imd

In [ ]:
if imd.max().all() > 0: 
    target = imd / imd.max()
else:
    target = imd / (imd.max() + 1)

In [ ]:
imd.max()

In [ ]:
target

In [ ]:
4.8/6.2

## assert two df are the same

In [ ]:
input_matrix = read_matrix("tests/resources/input_matrix_without_uncert.csv")
input_matrix = input_matrix.drop(input_matrix.columns[0],axis=1) 

In [ ]:
expected_res = read_matrix('tests/resources/normalization/res_minmax_no0.csv')

In [ ]:
norm = Normalization(input_matrix)
res = norm.minmax(feature_range=(0, 1))

In [ ]:
print(type(input_matrix))
print(type(expected_res))
print(type(res))

In [ ]:
res

In [ ]:
res.columns = res.columns.astype('str')

In [ ]:
res

In [ ]:
expected_res

In [ ]:
assert_frame_equal(res, expected_res, check_like=True)

In [ ]:
isinstance(res, pd.DataFrame)

In [ ]:
feature_range=(0, 1)